Imports


In [12]:
import torch
import torchvision
import pandas as pd
import torch.nn as nn
from tqdm import tqdm
import multiprocessing
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import tabulate

Versión torch


In [13]:
print("Torch version: ", torch.__version__)

####################################################################
# Set Device
####################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

Torch version:  2.5.1+cu121
Device:  cuda


Preparar datos


In [14]:
####################################################################
# Prepare Data
####################################################################

train_set = torchvision.datasets.MNIST(".data/", train=True, download=True)
# train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

test_set = torchvision.datasets.MNIST(".data/", train=False, download=True)
# test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True)

print("Train images: ", train_set)
print("Image: ", train_set[0][0])
print("Label: ", train_set[0][1])
print("Label one hot: ", F.one_hot(torch.tensor(train_set[0][1]), num_classes=10))

Train images:  Dataset MNIST
    Number of datapoints: 60000
    Root location: .data/
    Split: Train
Image:  <PIL.Image.Image image mode=L size=28x28 at 0x7C7CE048F0A0>
Label:  5
Label one hot:  tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])


Dataset class


In [15]:
####################################################################
# Dataset Class
####################################################################
class MNIST_dataset(Dataset):
    def __init__(self, data, partition="train"):
        print("\nLoading MNIST ", partition, " Dataset...")
        self.data = data
        self.partition = partition
        print("\tTotal Len.: ", len(self.data), "\n", 50 * "-")

    def __len__(self):
        return len(self.data)

    def from_pil_to_tensor(self, image):
        return torchvision.transforms.ToTensor()(image)

    def __getitem__(self, idx):
        # Image
        image = self.data[idx][0]
        # PIL Image to torch tensor
        image_tensor = self.from_pil_to_tensor(image)
        # care! net expect a 784 size vector and our dataset
        # provide 1x28x28 (channels, height, width) -> Reshape!
        image_tensor = image_tensor.view(-1)

        # Label
        label = torch.tensor(self.data[idx][1])
        label = F.one_hot(label, num_classes=10).float()

        return {"img": image_tensor, "label": label}


train_set = MNIST_dataset(train_set, partition="train")
test_set = MNIST_dataset(test_set, partition="test")


Loading MNIST  train  Dataset...
	Total Len.:  60000 
 --------------------------------------------------

Loading MNIST  test  Dataset...
	Total Len.:  10000 
 --------------------------------------------------


DataLoader class


In [16]:
####################################################################
# DataLoader Class
####################################################################

batch_size = 100
num_workers = multiprocessing.cpu_count() - 1
print("Num workers", num_workers)
train_dataloader = DataLoader(
    train_set, batch_size, shuffle=True, num_workers=num_workers
)
test_dataloader = DataLoader(
    test_set, batch_size, shuffle=False, num_workers=num_workers
)

Num workers 1


NN Class


In [17]:
####################################################################
# Neural Network Class
####################################################################
# Creating our Neural Network - Fully Connected
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(784, 256)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(256, 256)
        self.relu2 = nn.ReLU()
        self.linear3 = nn.Linear(256, 256)
        self.relu3 = nn.ReLU()
        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        out = self.relu1(self.linear1(x))
        out = self.relu2(self.linear2(out))
        out = self.relu3(self.linear3(out))
        out = self.classifier(out)
        return out


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

Función de entrenamineto


In [18]:
# TRAIN NETWORK
def train_network(optimizer, epoch, criterion, net):
    train_loss, train_correct = 0, 0
    net.train()
    with tqdm(
        iter(train_dataloader), desc="Epoch " + str(epoch), unit="batch"
    ) as tepoch:
        for batch in tepoch:
            # Returned values of Dataset Class
            images = batch["img"].to(device)
            labels = batch["label"].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # Forward
            outputs = net(images)
            loss = criterion(outputs, labels)

            # Calculate gradients
            loss.backward()

            # Update parameters
            optimizer.step()

            # one hot -> labels
            labels = torch.argmax(labels, dim=1)
            pred = torch.argmax(outputs, dim=1)
            train_correct += pred.eq(labels).sum().item()

            # print statistics
            train_loss += loss.item()

    train_loss /= len(train_dataloader.dataset)
    return train_loss, train_correct

Función de test


In [19]:
# TEST NETWORK
def test_network(net, epoch, criterion):
    test_loss, test_correct = 0, 0
    net.eval()
    with torch.no_grad():
        with tqdm(
            iter(test_dataloader), desc="Test " + str(epoch), unit="batch"
        ) as tepoch:
            for batch in tepoch:
                images = batch["img"].to(device)
                labels = batch["label"].to(device)

                # Forward
                outputs = net(images)
                test_loss += criterion(outputs, labels)

                # one hot -> labels
                labels = torch.argmax(labels, dim=1)
                pred = torch.argmax(outputs, dim=1)

                test_correct += pred.eq(labels).sum().item()

        test_loss /= len(test_dataloader.dataset)
        test_accuracy = 100.0 * test_correct / len(test_dataloader.dataset)
    return test_loss, test_correct, test_accuracy

Entrenamiento con validación


In [20]:
# Instantiating the network and printing its architecture
num_classes = 10
net = Net(num_classes)
print(net)
print("Params: ", count_parameters(net))
####################################################################
# Training settings
####################################################################

# Training hyperparameters
epochs = 15
criterion = nn.CrossEntropyLoss()
best_results = dict()
for lr in [0.1, 0.01, 0.001, 0.0001]:
    best_results[lr] = dict()
    for optimizer in [optim.AdamW, optim.SGD, optim.Adadelta, optim.Adam]:
        ####################################################################
        # Training
        ####################################################################

        # Load model in GPU, cada vez que hacemos un entrenamiento con un tipo de optimizador y lr tenemos que crear la red
        net = Net(num_classes)
        net.to(device)
        print("Params: ", count_parameters(net))
        # y el optimizador debe crearse despues de haber creado la red porque le pasamos los parámetros de la red que serán los que se encargue de optimizar
        if optimizer == optim.SGD:
            optimizer = optimizer(
                net.parameters(), lr=lr, weight_decay=1e-6, momentum=0.9
            )
        else:
            optimizer = optimizer(net.parameters(), lr=lr, weight_decay=1e-6)

        best_results[lr][optimizer.__class__.__name__] = {
            "best_accuracy": -1,
            "best_epoch": 0,
        }

        print("\n---- Start Training ----")
        print(f"Optimizer: {optimizer.__class__.__name__} - Learning Rate: {lr}")
        for epoch in range(epochs):
            train_loss, train_correct = train_network(optimizer, epoch, criterion, net)

            test_loss, test_correct, test_accuracy = test_network(net, epoch, criterion)

            print(
                "[Epoch {}] Train Loss: {:.6f} - Test Loss: {:.6f} - Train Accuracy: {:.2f}% - Test Accuracy: {:.2f}%".format(
                    epoch + 1,
                    train_loss,
                    test_loss,
                    100.0 * train_correct / len(train_dataloader.dataset),
                    test_accuracy,
                )
            )

            if (
                test_accuracy
                > best_results[lr][optimizer.__class__.__name__]["best_accuracy"]
            ):
                best_results[lr][optimizer.__class__.__name__][
                    "best_accuracy"
                ] = test_accuracy
                best_results[lr][optimizer.__class__.__name__]["best_epoch"] = epoch

                # Save best weights
                torch.save(net.state_dict(), "best_model.pt")

        print(
            "\nBEST TEST ACCURACY: ",
            best_results[lr][optimizer.__class__.__name__]["best_accuracy"],
            " in epoch ",
            best_results[lr][optimizer.__class__.__name__]["best_epoch"],
        )

####################################################################
# Results
####################################################################
print(best_results)

Net(
  (linear1): Linear(in_features=784, out_features=256, bias=True)
  (relu1): ReLU()
  (linear2): Linear(in_features=256, out_features=256, bias=True)
  (relu2): ReLU()
  (linear3): Linear(in_features=256, out_features=256, bias=True)
  (relu3): ReLU()
  (classifier): Linear(in_features=256, out_features=10, bias=True)
)
Params:  335114
Params:  335114

---- Start Training ----
Optimizer: AdamW - Learning Rate: 0.1


Test 0: 100%|██████████| 100/100 [00:02<00:00, 45.71batch/s]

[Epoch 1] Train Loss: 0.033213 - Test Loss: 0.018195 - Train Accuracy: 21.34% - Test Accuracy: 26.80%



Test 1: 100%|██████████| 100/100 [00:02<00:00, 38.90batch/s]

[Epoch 2] Train Loss: 0.017263 - Test Loss: 0.017299 - Train Accuracy: 29.56% - Test Accuracy: 29.60%


...

[Epoch 13] Train Loss: 0.000623 - Test Loss: 0.000861 - Train Accuracy: 98.17% - Test Accuracy: 97.45%



Test 13: 100%|██████████| 100/100 [00:02<00:00, 37.78batch/s]

[Epoch 14] Train Loss: 0.000553 - Test Loss: 0.000852 - Train Accuracy: 98.39% - Test Accuracy: 97.58%



Test 14: 100%|██████████| 100/100 [00:02<00:00, 44.92batch/s]

[Epoch 15] Train Loss: 0.000500 - Test Loss: 0.000810 - Train Accuracy: 98.55% - Test Accuracy: 97.68%

BEST TEST ACCURACY:  97.68  in epoch  14
{0.1: {'AdamW': {'best_accuracy': 35.49, 'best_epoch': 6}, 'SGD': {'best_accuracy': 98.11, 'best_epoch': 12}, 'Adadelta': {'best_accuracy': 97.87, 'best_epoch': 14}, 'Adam': {'best_accuracy': 11.35, 'best_epoch': 7}}, 0.01: {'AdamW': {'best_accuracy': 97.35, 'best_epoch': 10}, 'SGD': {'best_accuracy': 98.08, 'best_epoch': 11}, 'Adadelta': {'best_accuracy': 92.3, 'best_epoch': 14}, 'Adam': {'best_accuracy': 97.59, 'best_epoch': 14}}, 0.001: {'AdamW': {'best_accuracy': 98.1, 'best_epoch': 14}, 'SGD': {'best_accuracy': 94.34, 'best_epoch': 14}, 'Adadelta': {'best_accuracy': 61.55, 'best_epoch': 14}, 'Adam': {'best_accuracy': 98.2, 'best_epoch': 14}}, 0.0001: {'AdamW': {'best_accuracy': 97.69, 'best_epoch': 13}, 'SGD': {'best_accuracy': 53.0, 'best_epoch': 14}, 'Adadelta': {'best_accuracy': 14.47, 'best_epoch': 14}, 'Adam': {'best_accuracy': 97.68

In [21]:
print(best_results)

{0.1: {'AdamW': {'best_accuracy': 35.49, 'best_epoch': 6}, 'SGD': {'best_accuracy': 98.11, 'best_epoch': 12}, 'Adadelta': {'best_accuracy': 97.87, 'best_epoch': 14}, 'Adam': {'best_accuracy': 11.35, 'best_epoch': 7}}, 0.01: {'AdamW': {'best_accuracy': 97.35, 'best_epoch': 10}, 'SGD': {'best_accuracy': 98.08, 'best_epoch': 11}, 'Adadelta': {'best_accuracy': 92.3, 'best_epoch': 14}, 'Adam': {'best_accuracy': 97.59, 'best_epoch': 14}}, 0.001: {'AdamW': {'best_accuracy': 98.1, 'best_epoch': 14}, 'SGD': {'best_accuracy': 94.34, 'best_epoch': 14}, 'Adadelta': {'best_accuracy': 61.55, 'best_epoch': 14}, 'Adam': {'best_accuracy': 98.2, 'best_epoch': 14}}, 0.0001: {'AdamW': {'best_accuracy': 97.69, 'best_epoch': 13}, 'SGD': {'best_accuracy': 53.0, 'best_epoch': 14}, 'Adadelta': {'best_accuracy': 14.47, 'best_epoch': 14}, 'Adam': {'best_accuracy': 97.68, 'best_epoch': 14}}}


In [22]:
# mostrar resultats del diccionari amb tabulate
for lr in best_results:
    for optimizer in best_results[lr]:
        best_results[lr][optimizer] = (
            round(best_results[lr][optimizer]["best_accuracy"], 2),
            best_results[lr][optimizer]["best_epoch"],
        )

headers = best_results[list(best_results.keys())[0]].keys()
values = [[name, *inner.values()] for name, inner in best_results.items()]

print(tabulate.tabulate(values, headers=headers))


best_accuracy = -1
best_lr = None
best_optimizer = None
for lr in best_results:
    for optimizer in best_results[lr]:
        if best_results[lr][optimizer][0] > best_accuracy:
            best_accuracy = best_results[lr][optimizer][0]
            best_lr = lr
            best_optimizer = optimizer

print(
    "\nBEST TEST ACCURACY: ",
    best_accuracy,
    " with lr: ",
    best_lr,
    " and optimizer: ",
    best_optimizer,
)
####################################################################
# Load best weights
####################################################################

# Load best weights, tenemos que tener una red creada con la misma arquitectura que la que hemos entrenado
net = Net(num_classes)
net.load_state_dict(torch.load("best_model.pt"))

_, _, test_accuracy = test_network(net, 0, criterion)
print("\nFinal best acc: ", test_accuracy)

        AdamW        SGD          Adadelta     Adam
------  -----------  -----------  -----------  -----------
0.1     (35.49, 6)   (98.11, 12)  (97.87, 14)  (11.35, 7)
0.01    (97.35, 10)  (98.08, 11)  (92.3, 14)   (97.59, 14)
0.001   (98.1, 14)   (94.34, 14)  (61.55, 14)  (98.2, 14)
0.0001  (97.69, 13)  (53.0, 14)   (14.47, 14)  (97.68, 14)

BEST TEST ACCURACY:  98.2  with lr:  0.001  and optimizer:  Adam


<ipython-input-22-e80b439939e6>:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("best_model.pt"))
Test 0:   0%|          | 0/100 [00:00<?, ?

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)